<a href="https://colab.research.google.com/github/SamiraHabib/Coursera_Capstone/blob/main/Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursera Capstone #

# Importando as bibliotecas #

In [61]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


# Definição das credenciais e version do Foursquare #

In [62]:
CLIENT_ID = '1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA' # your Foursquare ID
CLIENT_SECRET = 'ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA
CLIENT_SECRET:ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D


# Endereço do local escolhido #

In [184]:
#Exemplo - alterar
address = 'Rio de Janeiro, RJ'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-22.9110137 -43.2093727


# Localização #

In [154]:
#Localidade exemplo, mudar
latitude = -22.9110137
longitude = -43.2093727

# Define URL #

In [168]:
search_query = 'escola'
radius = 1500
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

escola .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA&client_secret=ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D&ll=-22.9110137,-43.2093727&v=20180604&query=escola&radius=1500&limit=50'

# Envia uma solicitação GET e examina os resultados #

In [169]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6081e2e5126a804c34e2e01a'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
       'suffix': '.png'},
      'id': '52e81612bcbc57f1066b7a43',
      'name': 'Circus School',
      'pluralName': 'Circus Schools',
      'primary': True,
      'shortName': 'Circus School'}],
    'hasPerk': False,
    'id': '53e37ca3498e9b349be89e5b',
    'location': {'address': 'Pç. da Bandeira, 4',
     'cc': 'BR',
     'city': 'Rio de Janeiro',
     'country': 'Brasil',
     'distance': 352,
     'formattedAddress': ['Pç. da Bandeira, 4',
      'Rio de Janeiro, RJ',
      '20270-150',
      'Brasil'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -22.9103070588052,
       'lng': -43.21272296036773}],
     'lat': -22.9103070588052,
     'lng': -43.21272296036773,
     'postalCode': '20270-150',
     'state': 'RJ'},
    'name': 'Escola Nacional de Circo (ENC)',
    'referralId': 'v-161

## Pegando a parte relevante do JSON e transformando em um dataframe (pandas) ##

In [175]:
#transformando em dataframe
venues = results['response']['venues']
df_school = json_normalize(venues)
df_school.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,53e37ca3498e9b349be89e5b,Escola Nacional de Circo (ENC),"[{'id': '52e81612bcbc57f1066b7a43', 'name': 'C...",v-1619124965,False,"Pç. da Bandeira, 4",-22.910307,-43.212723,"[{'label': 'display', 'lat': -22.9103070588052...",352,20270-150,BR,Rio de Janeiro,RJ,Brasil,"[Pç. da Bandeira, 4, Rio de Janeiro, RJ, 20270...",NaN
1,526324aa498e769d346bee7a,EDCM - Escola de Dança Caio Monatte,"[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",v-1619124965,False,"R. Haddock Lobo, 79",-22.916425,-43.209381,"[{'label': 'display', 'lat': -22.9164253712554...",602,20260-130,BR,Rio de Janeiro,RJ,Brasil,"[R. Haddock Lobo, 79, Rio de Janeiro, RJ, 2026...",NaN
2,5399a301498ea6a82319015b,Escola de Enfermagem Anna Nery,"[{'id': '4bf58dd8d48988d1a8941735', 'name': 'G...",v-1619124965,False,"R. Afonso Cavalcanti, 275",-22.909833,-43.201483,"[{'label': 'display', 'lat': -22.9098327961144...",819,NaN,BR,Rio de Janeiro,RJ,Brasil,"[R. Afonso Cavalcanti, 275, Rio de Janeiro, R...",NaN
3,4fce2f5de4b0155a2eafa4dd,Escola de Circo Crescer e Viver,"[{'id': '52e81612bcbc57f1066b7a43', 'name': 'C...",v-1619124965,False,"R. Carmo Neto, 143",-22.911806,-43.198873,"[{'label': 'display', 'lat': -22.9118055514465...",1080,20210-051,BR,Rio de Janeiro,RJ,Brasil,"[R. Carmo Neto, 143, Rio de Janeiro, RJ, 20210...",NaN
4,4f4b6c2be4b08334fd32bf44,Hospital Escola São Francisco de Assis,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1619124965,False,"Av. Presidente Vargas, 2863",-22.909555,-43.201416,"[{'label': 'display', 'lat': -22.9095548564292...",831,20210-030,BR,Rio de Janeiro,RJ,Brasil,"[Av. Presidente Vargas, 2863, Rio de Janeiro, ...",NaN


# Define information of interest and filter dataframe #

In [177]:
# Mantém somente as colunas que incluem venue name e qualquer coisa que é associada com a localização
filtered_columns = ['name', 'categories'] + [col for col in df_school.columns if col.startswith('location.')] + ['id']
df_school_filtered = df_school.loc[:, filtered_columns]

# função que extrai a categoria de venue 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filtra a categoria por cada linha do data frame
df_school_filtered['categories'] = df_school_filtered.apply(get_category_type, axis=1)

# limpa os nomes das colunas para manter somente o último termo
df_school_filtered.columns = [column.split('.')[-1] for column in df_school_filtered.columns]

df_school_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Escola Nacional de Circo (ENC),Circus School,"Pç. da Bandeira, 4",-22.910307,-43.212723,"[{'label': 'display', 'lat': -22.9103070588052...",352,20270-150,BR,Rio de Janeiro,RJ,Brasil,"[Pç. da Bandeira, 4, Rio de Janeiro, RJ, 20270...",NaN,53e37ca3498e9b349be89e5b
1,EDCM - Escola de Dança Caio Monatte,Dance Studio,"R. Haddock Lobo, 79",-22.916425,-43.209381,"[{'label': 'display', 'lat': -22.9164253712554...",602,20260-130,BR,Rio de Janeiro,RJ,Brasil,"[R. Haddock Lobo, 79, Rio de Janeiro, RJ, 2026...",NaN,526324aa498e769d346bee7a
2,Escola de Enfermagem Anna Nery,General College & University,"R. Afonso Cavalcanti, 275",-22.909833,-43.201483,"[{'label': 'display', 'lat': -22.9098327961144...",819,NaN,BR,Rio de Janeiro,RJ,Brasil,"[R. Afonso Cavalcanti, 275, Rio de Janeiro, R...",NaN,5399a301498ea6a82319015b
3,Escola de Circo Crescer e Viver,Circus School,"R. Carmo Neto, 143",-22.911806,-43.198873,"[{'label': 'display', 'lat': -22.9118055514465...",1080,20210-051,BR,Rio de Janeiro,RJ,Brasil,"[R. Carmo Neto, 143, Rio de Janeiro, RJ, 20210...",NaN,4fce2f5de4b0155a2eafa4dd
4,Hospital Escola São Francisco de Assis,Hospital,"Av. Presidente Vargas, 2863",-22.909555,-43.201416,"[{'label': 'display', 'lat': -22.9095548564292...",831,20210-030,BR,Rio de Janeiro,RJ,Brasil,"[Av. Presidente Vargas, 2863, Rio de Janeiro, ...",NaN,4f4b6c2be4b08334fd32bf44


# Tipo de dados, nome e categorias dos dados #

In [187]:
df_school_filtered.dtypes

name                 object
categories           object
address              object
lat                 float64
lng                 float64
labeledLatLngs       object
distance              int64
postalCode           object
cc                   object
city                 object
state                object
country              object
formattedAddress     object
crossStreet          object
id                   object
dtype: object

In [178]:
df_school_filtered.name

0                    Escola Nacional de Circo (ENC)
1               EDCM - Escola de Dança Caio Monatte
2                    Escola de Enfermagem Anna Nery
3                   Escola de Circo Crescer e Viver
4            Hospital Escola São Francisco de Assis
5      Escola Espírita de Evangelho Gamaliel (EEEG)
6                              Escola de Engenharia
7                   Escola de Natação Brilho D'água
8     Escola Municipal Reverendo Martin Luther King
9                            Escola de Defesa Civil
10                      Escola Diaconal Santo Efrém
11                   Escola Municipal Azevedo Sodre
12                                   Escola Vitória
13                             Escola Mario Claudio
14            Escola de Enfermagem Anna Nery - UFRJ
15                              Escola de Modelagem
16                   Escola De Enfermagem Anna Nery
17                     Escola de Vôlei Camila Braga
18                                      Escola Jóia
19          

In [179]:
df_school_filtered.categories

0                    Circus School
1                     Dance Studio
2     General College & University
3                    Circus School
4                         Hospital
5                 Religious School
6     College Engineering Building
7                      Swim School
8                     Trade School
9                     Fire Station
10                Spiritual Center
11                   Middle School
12               Elementary School
13                       City Hall
14            College & University
15                    Trade School
16                  Student Center
17                Volleyball Court
18                     Event Space
19    General College & University
20               Elementary School
21            Gym / Fitness Center
22                  Student Center
23    College Engineering Building
24                          School
25                  Student Center
26              Athletics & Sports
27                  Student Center
28                  

In [180]:
df_school_filtered.columns

Index(['name', 'categories', 'address', 'lat', 'lng', 'labeledLatLngs',
       'distance', 'postalCode', 'cc', 'city', 'state', 'country',
       'formattedAddress', 'crossStreet', 'id'],
      dtype='object')

# Identificar as ids de escolas diferentes #

In [182]:
df_school_filtered[['id', 'name']]

,id,name
0,53e37ca3498e9b349be89e5b,Escola Nacional de Circo (ENC)
1,526324aa498e769d346bee7a,EDCM - Escola de Dança Caio Monatte
2,5399a301498ea6a82319015b,Escola de Enfermagem Anna Nery
3,4fce2f5de4b0155a2eafa4dd,Escola de Circo Crescer e Viver
4,4f4b6c2be4b08334fd32bf44,Hospital Escola São Francisco de Assis
5,59998b322ebb3418ae5f09f7,Escola Espírita de Evangelho Gamaliel (EEEG)
6,52693fdf11d26f25c17f8299,Escola de Engenharia
7,4e4595a6fa76a07fde4e2148,Escola de Natação Brilho D'água
8,4f16a808e4b0380207d3d120,Escola Municipal Reverendo Martin Luther King
9,5579674e498ebe4bbf25b1eb,Escola de Defesa Civil


# Visualizar escolas no mapa ao redor da localização anteriormente citada #

In [183]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # gerar mapa centrado no entorno do RJ

# Circulo vermelho no centro
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='School',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# Escolas em azul ao entorno
for lat, lng, label in zip(df_school_filtered.lat, df_school_filtered.lng, df_school_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
       # popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map